In [1]:
 #import libraries
# ================

# for date and time opeations
from datetime import datetime
# for file and folder operations
import os
# for regular expression opeations
import re
# for listing files in a folder
import glob
# for getting web contents
import requests 
# storing and analysing data
import pandas as pd
# for scraping web contents
from bs4 import BeautifulSoup
# regular expression
import re
# numerical analysis
import numpy as np

In [2]:
# get data
# ========

# link at which web data recides
link = 'https://www.worldometers.info/coronavirus/'
# get web data
req = requests.get(link)
# parse web data
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
# find the table
# ==============
# our target table is the last table in the page

# get the table head
# table head may contain the column names, titles, subtitles
thead = soup.find_all('thead')[-1]
# print(thead)

# get all the rows in table head
# it usually have only one row, which has the column names
head = thead.find_all('tr')
# print(head)

# get the table tbody
# it contains the contents
tbody = soup.find_all('tbody')[0]
# print(tbody)

# get all the rows in table body
# each row is each state's entry
body = tbody.find_all('tr')
# print(body)

In [4]:
# get the table contents
# ======================

# container for header rows / column title
head_rows = []
# container for table body / contents
body_rows = []

# loop through the head and append each row to head
for tr in head:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    head_rows.append(row)
# print(head_rows)

# loop through the body and append each row to body
for tr in body:
    td = tr.find_all(['th', 'td'])
    row = [i.text for i in td]
    body_rows.append(row)
# print(head_rows)

In [5]:
# save contents in a dataframe
# ============================
    
# skip last 3 rows, it contains unwanted info
# head_rows contains column title
df_bs = pd.DataFrame(body_rows[:len(body_rows)-6], 
                     columns=head_rows[0])         

# Drop 'S. No.' column
# df_bs.drop('S. No.', axis=1, inplace=True)

df_bs.head(5)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",...,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl,New Cases/1M pop,New Deaths/1M pop,Active Cases/1M pop
0,,\nNorth America\n,"41,057,755","+3,573","925,153",+124,"34,513,298","+10,088","5,619,304","11,379",...,,,,North America,\n,,,,,
1,,\nAsia\n,"57,937,057","+139,725","825,498","+1,770","54,945,569","+91,204","2,165,990","28,718",...,,,,Asia,\n,,,,,
2,,\nSouth America\n,"34,070,047","+1,195","1,038,793",+31,"31,398,365","+1,445","1,632,889","28,041",...,,,,South America,\n,,,,,
3,,\nEurope\n,"49,058,901","+40,294","1,114,827",+868,"45,940,302","+28,323","2,003,772","6,140",...,,,,Europe,\n,,,,,
4,,\nAfrica\n,"6,038,873","+2,994","152,832",+7,"5,253,747",+480,"632,294","4,356",...,,,,Africa,\n,,,,,


In [6]:
# drop unwanted rows
df_bs = df_bs.iloc[8:, :-6].reset_index(drop=True)

# drop unwanted columns
df_bs = df_bs.drop('#', axis=1)

# first few rows
df_bs.head()

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent
0,USA,"34,766,404",,"623,029",,"29,274,349",,"4,869,026","4,457","104,404","1,871","513,059,613","1,540,729","332,997,863",North America
1,India,"30,907,282","+2,548","409,338",,"30,063,720","+7,590","434,224","8,944","22,173",294,"434,058,138","311,390","1,393,938,700",Asia
2,Brazil,"19,106,971",,"534,311",,"17,666,654",,"906,006","8,318","89,238","2,495","54,204,686","253,160","214,112,172",South America
3,Russia,"5,833,175","+24,702","144,492",+780,"5,236,214","+19,566","452,469","2,300","39,954",990,"156,300,000","1,070,556","145,998,857",Europe
4,France,"5,813,899",,"111,353",,"5,648,649",,"53,897",957,"88,868","1,702","96,853,814","1,480,445","65,422,093",Europe


In [7]:
# rename columns
df_bs.columns = ['Country/Region', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop',
       'Population', 'Continent']

# rearrange and subselect columns
df_bs = df_bs[['Country/Region', 'Continent', 'Population', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths',
       'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical',
       'Tot Cases/1M pop', 'Deaths/1M pop', 'TotalTests', 'Tests/1M pop' ]]

# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop
0,USA,North America,"332,997,863","34,766,404",,"623,029",,"29,274,349",,"4,869,026","4,457","104,404","1,871","513,059,613","1,540,729"
1,India,Asia,"1,393,938,700","30,907,282","+2,548","409,338",,"30,063,720","+7,590","434,224","8,944","22,173",294,"434,058,138","311,390"
2,Brazil,South America,"214,112,172","19,106,971",,"534,311",,"17,666,654",,"906,006","8,318","89,238","2,495","54,204,686","253,160"
3,Russia,Europe,"145,998,857","5,833,175","+24,702","144,492",+780,"5,236,214","+19,566","452,469","2,300","39,954",990,"156,300,000","1,070,556"
4,France,Europe,"65,422,093","5,813,899",,"111,353",,"5,648,649",,"53,897",957,"88,868","1,702","96,853,814","1,480,445"


In [8]:
who_region = {}

# African Region AFRO
afro = "Algeria, Angola, Cabo Verde, Congo, DRC, Eswatini, Sao Tome and Principe, Benin, South Sudan, Western Sahara, Congo (Brazzaville), Congo (Kinshasa), Cote d'Ivoire, Botswana, Burkina Faso, Burundi, Cameroon, Cape Verde, Central African Republic, Chad, Comoros, Ivory Coast, Democratic Republic of the Congo, Equatorial Guinea, Eritrea, Ethiopia, Gabon, Gambia, Ghana, Guinea, Guinea-Bissau, Kenya, Lesotho, Liberia, Madagascar, Malawi, Mali, Mauritania, Mauritius, Mozambique, Namibia, Niger, Nigeria, Republic of the Congo, Rwanda, São Tomé and Príncipe, Senegal, Seychelles, Sierra Leone, Somalia, South Africa, Swaziland, Togo, Uganda, Tanzania, Zambia, Zimbabwe"
afro = [i.strip() for i in afro.split(',')]
for i in afro:
    who_region[i] = 'Africa'
    
# Region of the Americas PAHO
paho = 'Antigua and Barbuda, Argentina, Bahamas, Barbados, Belize, Bermuda, Bolivia, Brazil, Canada, Chile, Colombia, Costa Rica, Cuba, Dominica, Dominican Republic, Ecuador, El Salvador, Grenada, Guatemala, Guyana, Haiti, Honduras, Jamaica, Mexico, Nicaragua, Panama, Paraguay, Peru, Saint Kitts and Nevis, Saint Lucia, Saint Vincent and the Grenadines, Suriname, Trinidad and Tobago, United States, US, USA, Uruguay, Venezuela'
paho = [i.strip() for i in paho.split(',')]
for i in paho:
    who_region[i] = 'Americas'

# South-East Asia Region SEARO
searo = 'Bangladesh, Bhutan, North Korea, India, Indonesia, Maldives, Myanmar, Burma, Nepal, Sri Lanka, Thailand, Timor-Leste'
searo = [i.strip() for i in searo.split(',')]
for i in searo:
    who_region[i] = 'South-East Asia'

# European Region EURO
euro = 'Albania, Andorra, Greenland, Kosovo, Holy See, Vatican City, Liechtenstein, Armenia, Czechia, Austria, Azerbaijan, Belarus, Belgium, Bosnia and Herzegovina, Bulgaria, Croatia, Cyprus, Czech Republic, Denmark, Estonia, Finland, France, Georgia, Germany, Greece, Hungary, Iceland, Ireland, Israel, Italy, Kazakhstan, Kyrgyzstan, Latvia, Lithuania, Luxembourg, Malta, Monaco, Montenegro, Netherlands, North Macedonia, Norway, Poland, Portugal, Moldova, Romania, Russia, San Marino, Serbia, Slovakia, Slovenia, Spain, Sweden, Switzerland, Tajikistan, Turkey, Turkmenistan, Ukraine, United Kingdom, UK, Uzbekistan'
euro = [i.strip() for i in euro.split(',')]
for i in euro:
    who_region[i] = 'Europe'
# Eastern Mediterranean Region EMRO
emro = 'Afghanistan, Bahrain, Djibouti, Egypt, Iran, Iraq, Jordan, Kuwait, Lebanon, Libya, Morocco, Oman, Pakistan, Palestine, West Bank and Gaza, Qatar, Saudi Arabia, Somalia, Sudan, Syria, Tunisia, United Arab Emirates, UAE, Yemen'
emro = [i.strip() for i in emro.split(',')]
for i in emro:
    who_region[i] = 'Eastern Mediterranean'

# Western Pacific Region WPRO
wpro = 'Australia, Brunei, Cambodia, China, Cook Islands, Fiji, Japan, Hong Kong, Kiribati, Laos, Malaysia, Marshall Islands, Micronesia, Mongolia, Nauru, New Zealand, Niue, Palau, Papua New Guinea, Philippines, South Korea, S. Korea, Samoa, Singapore, Solomon Islands, Taiwan, Taiwan*, Tonga, Tuvalu, Vanuatu, Vietnam'
wpro = [i.strip() for i in wpro.split(',')]
for i in wpro:
    who_region[i] = 'Western Pacific'

In [9]:
# addding 'WHO Region' column
df_bs['WHO Region'] = df_bs['Country/Region'].map(who_region)

# countries with missing "WHO Region"
df_bs[df_bs['WHO Region'].isna()]['Country/Region'].unique()

array(['Réunion', 'French Guiana', 'Mayotte', 'French Polynesia',
       'Guadeloupe', 'Curaçao', 'Martinique', 'Aruba', 'CAR',
       'Channel Islands', 'Gibraltar', 'Sint Maarten', 'Turks and Caicos',
       'Saint Martin', 'St. Vincent Grenadines', 'Isle of Man',
       'Caribbean Netherlands', 'British Virgin Islands', 'St. Barth',
       'Faeroe Islands', 'Diamond Princess', 'Cayman Islands',
       'Wallis and Futuna', 'Brunei ', 'New Caledonia', 'Anguilla',
       'Falkland Islands', 'Macao', 'Saint Pierre Miquelon', 'Montserrat',
       'MS Zaandam'], dtype=object)

In [10]:
# fix data
for col in df_bs.columns[2:]:
    # replace comma with empty string
    df_bs[col] = df_bs[col].str.replace('[,+ ]', '', regex=True)
    # replace 'N/A' with empty string
    df_bs[col] = df_bs[col].str.replace('N/A', '', regex=False)

# replace empty strings with np.nan
df_bs = df_bs.replace('', np.nan)

In [11]:
# first few rows
df_bs.head()

,Country/Region,Continent,Population,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/1M pop,WHO Region
0,USA,North America,332997863,34766404,NaN,623029,NaN,29274349,NaN,4869026,4457,104404,1871,513059613,1540729,Americas
1,India,Asia,1393938700,30907282,2548,409338,NaN,30063720,7590,434224,8944,22173,294,434058138,311390,South-EastAsia
2,Brazil,South America,214112172,19106971,NaN,534311,NaN,17666654,NaN,906006,8318,89238,2495,54204686,253160,Americas
3,Russia,Europe,145998857,5833175,24702,144492,780,5236214,19566,452469,2300,39954,990,156300000,1070556,Europe
4,France,Europe,65422093,5813899,NaN,111353,NaN,5648649,NaN,53897,957,88868,1702,96853814,1480445,Europe


In [12]:
# save as .csv file
df_bs.to_csv('worldometer_data.csv', index=False)